In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
import datetime
warnings.filterwarnings('ignore')
warnings.warn('this will not show')
sns.set(style='darkgrid')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/cleaned.pickle')


In [ ]:
df = data.copy()
df_origin=data.copy()

In [ ]:
df

,location,province,make,model,make_model,price,body_type,registration,vehicle_age,mileage,...,engine_size,Fuel_type,upholstery_colour,doors,upholstery,Entertainment & Media,Comfort & Convenience,Safety,extras,general_inspection
0,loon op zand,North Brabant,Audi,A3,Audi_A3,4450,Compact,2003,19.0,191066,...,1984,Super 95,Black,3,Cloth,"CD player, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...",Alloy wheels,05/2022
1,heerlen,Limburg,Audi,A3,Audi_A3,12950,Compact,2013,9.0,94758,...,1395,Gasoline,Black,5,Cloth,"CD player, Radio","Air conditioning, Automatic climate control, E...","[['Alarm system', 'Central door lock', 'Centra...",Alloy wheels,None
2,breda,North Brabant,Audi,A3,Audi_A3,12445,Compact,2013,9.0,160883,...,1395,Gasoline,None,5,None,"Bluetooth, CD player, Hands-free equipment, MP...","Air conditioning, Armrest, Cruise control, Ele...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Spoiler",03/2022
3,baarle-nassau,North Brabant,Audi,A1,Audi_A1,12450,Compact,2015,7.0,145102,...,1395,Gasoline,None,5,None,"Bluetooth, CD player, On-board computer, USB","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock with remote contro...",Alloy wheels,New
4,zutphen,Gelderland,Audi,A1,Audi_A1,14945,Compact,2015,7.0,119666,...,1395,Gasoline,Black,3,Part leather,"CD player, MP3, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Bi-Xenon headlights'...","Alloy wheels,Shift paddles,Sport seats,Sport s...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71089,hendrik-ido-ambacht,South Holland,Volvo,S60,Volvo_S60,1999,Sedan,2005,17.0,442713,...,2435,LPG,None,4,Part leather,"CD player, On-board computer, Radio","Air conditioning, Automatic climate control, C...","[['ABS', 'Central door lock', 'Central door lo...",Alloy wheels,New
71090,kampen,Friesland,Volvo,S60,Volvo_S60,3950,Sedan,2005,17.0,251723,...,2435,Liquid petroleum gas (LPG),Beige,4,Full leather,Radio,"Air conditioning, Automatic climate control, C...","[['ABS', 'Alarm system', 'Central door lock', ...",Alloy wheels,04/2022
71091,lelystad,Flevoland,Volvo,S60,Volvo_S60,3449,Sedan,2004,18.0,340257,...,1984,LPG,None,4,Part leather,On-board computer,"Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Automatically dimming interior mi...",New
71092,amsterdam,North Holland,Volvo,S40,Volvo_S40,1650,Sedan,2004,18.0,399990,...,,Liquid petroleum gas (LPG),Beige,4,Velour,"CD player, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock with remote contro...","Alloy wheels,Automatically dimming interior mi...",None


### Functions

In [ ]:
def fill(df,col_name, methodd,*args):
  groups = list(args)
  if methodd == 'median':  
    df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform(methodd))
    if df[col_name].isna().sum() != 0 : 
      df[col_name] = df[col_name].fillna(df.groupby(groups[0])[col_name].transform(methodd)).fillna(df.groupby(groups[1])[col_name].transform(methodd))
      print('last Nan values filled only group1')
      if df[col_name].isna().sum() != 0 :
        df[col_name] = df[col_name].fillna(df[col_name].median())
        print('last Nan values filled with columns')
        
  elif methodd =='mode':
    fill_most(df,col_name,*args)

  elif methodd == 'bfill' : 
    df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform('bfill'))
    df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform('ffill'))
    if df[col_name].isna().sum() != 0 :
      df[col_name] = df[col_name].fillna(method =methodd)
      print('last Nan values filled with columns')
  elif methodd =='ffill':
    df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform('ffill')) 
    df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform('bfill'))
    if df[col_name].isna().sum() != 0 :
      df[col_name] = df[col_name].fillna(method =methodd)
      print('last Nan values filled with columns')
       
  print("Number of NaN : ",df[col_name].isnull().sum())
  print("------------------")
  print(df[col_name].value_counts(dropna=False))

# Grup Sutununa(group_col) gore gruplayip, uzerinde calisilan sutunun(col_name) null degerlerini ffill & bfill ile doldurur.
# Hala null deger kalma ihtimaline binaen, gruplama yapmaksizin, uzerinde calisilan sutunda da(col_name) ekstradan ffill & bfill yapar
def fill_prop(df, col_name, *args):
  groups = list(args)
  df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform('ffill')).fillna(df.groupby(groups)[col_name].transform('bfill'))
  if df[col_name].isna().sum() != 0 : 
    df[col_name] = df[col_name].fillna(df.groupby(groups)[col_name].transform('bfill')).fillna(df.groupby(groups)[col_name].transform('ffill'))
  print("Number of NaN : ",df[col_name].isnull().sum())
  print("------------------")
  print(df[col_name].value_counts(dropna=False))

# Grup Sutununa(group_col) gore gruplayip, uzerinde calisilan sutunun(col_name) null degerlerini mode ile doldurur.
def fill_most(df,col_name, *args ):
  groups = list(args)
  df[col_name] = df.groupby(groups)[col_name].apply(lambda x: x.fillna(x.mode().iat[0]) if x.notna().any() else x)

  if df[col_name].isna().sum() != 0 :
    df[col_name] = df.groupby(groups[0])[col_name].apply(lambda x: x.fillna(x.mode().iat[0]) if x.notna().any() else x)
    print('last Nan values filled only group1')
    if df[col_name].isna().sum() != 0 :
      df[col_name] = df[col_name].fillna(df[col_name].mode()[0])
      print('last Nan values filled with columns')
  print("Number of NaN : ",df[col_name].isnull().sum())
  print("------------------")
  print(df[col_name].value_counts(dropna=False))

3 body_type

In [ ]:
df.replace({'None': np.nan , 'nan':np.nan},inplace = True)

In [ ]:
df.body_type.isnull().sum()

1074

In [ ]:
fill(df, 'body_type', 'ffill','make', 'make_model')

Number of NaN :  0
------------------
Off-Road/Pick-up    19750
Compact             15173
Station wagon       13063
Van                  7171
Sedan                6243
Convertible          4116
Transporter          2627
Coupe                2225
Other                 726
Name: body_type, dtype: int64


In [ ]:
df.body_type.isnull().sum()

0

### price ,engine_size anomalies

In [ ]:
#drop rows where price=NaN from dataframe,afterwards reset index ,convert object to int
#additionally,some anomalies converted in np.nan

In [ ]:
df.price.replace({'': np.nan},inplace = True)

In [ ]:
df["price"]=df["price"].astype(float)

In [ ]:
df.dropna(subset = ["price"], inplace=True)

In [ ]:
df["price"]=df["price"].astype(int)

In [ ]:
df['price'] = np.where((df.price>400000),np.nan,df.price)
df['price'] = np.where((df.price<100),np.nan,df.price)

In [ ]:
df["price"].isna().sum()

10

In [ ]:
df.dropna(subset = ["price"], inplace=True)

In [ ]:
df.engine_size.replace({'': np.nan},inplace = True)

In [ ]:
df["engine_size"].isna().sum()

5744

In [ ]:
fill_most(df,"engine_size","make_model")

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
1598    5645
1998    2931
1199    2882
999     2577
1395    2367
        ... 
6466       1
3556       1
3495       1
7440       1
200        1
Name: engine_size, Length: 454, dtype: int64


In [ ]:
fill_most(df,"engine_size","make_model",'cylinders')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
1598    5750
1998    2940
1199    2885
999     2575
1395    2402
        ... 
2293       1
2156       1
8684       1
3179       1
200        1
Name: engine_size, Length: 445, dtype: int64


In [ ]:
df["engine_size"]= df["engine_size"].astype(int)

In [ ]:
df['engine_size'] = np.where((df.engine_size>10000),np.nan,df.engine_size)

In [ ]:
df["engine_size"].isna().sum()

2

In [ ]:
df.dropna(subset = ["price"], inplace=True)

In [ ]:
df[df["engine_size"]<500.0].value_counts(dropna=False)

location       province       make        model           make_model           price    body_type         registration  vehicle_age  mileage  Gearbox    gears  seller  paint      colour  power(kW)  cylinders  drivetrain  type            previous_owner  full_service_history  fuel_type  combination(L/100Km)  city(L/100Km)  country(L/100Km)  non_smoker_vehicle  empty_weight  co2_emissions  emission_class(Euro)  seats  warranty_months  engine_size  Fuel_type  upholstery_colour  doors  upholstery    Entertainment & Media                                                                                        Comfort & Convenience                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
df['engine_size'] = np.where((df.engine_size<500),np.nan,df.engine_size)

In [ ]:
df["engine_size"].isna().sum()

2756

In [ ]:
fill_most(df,"engine_size","make_model")

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
1598.0    8484
1998.0    2943
1199.0    2890
999.0     2575
1395.0    2402
          ... 
1985.0       1
3940.0       1
3474.0       1
3664.0       1
2499.0       1
Name: engine_size, Length: 433, dtype: int64


In [ ]:
fill_most(df,"engine_size","make_model",'cylinders')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
1598.0    8484
1998.0    2943
1199.0    2890
999.0     2575
1395.0    2402
          ... 
1985.0       1
3940.0       1
3474.0       1
3664.0       1
2499.0       1
Name: engine_size, Length: 433, dtype: int64


In [ ]:
df["engine_size"].isna().sum()

0

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df["price"]=df["price"].astype(int)

In [ ]:
df["engine_size"]=df["engine_size"].astype(int)

In [ ]:
df["price"]

0         4450
1        12950
2        12445
3        12450
4        14945
         ...  
69509     1999
69510     3950
69511     3449
69512     1650
69513      944
Name: price, Length: 69514, dtype: int64

In [ ]:
df["engine_size"]

0        1984
1        1395
2        1395
3        1395
4        1395
         ... 
69509    2435
69510    2435
69511    1984
69512    2435
69513    2435
Name: engine_size, Length: 69514, dtype: int64

## OMER FARUK

model


In [ ]:
df.model.isna().sum()

0

In [ ]:
df.model.unique()

array(['A3', 'A1', 'A5', 'A7', 'S5', 'Others', 'TT', 'A4', 'R8', 'Q5',
       'Q3', 'Q2', 'Q7', 'A6 allroad', 'A4 allroad', 'SQ7', 'SQ5', 'A6',
       'A8', 'Q8', 'e-tron GT', 'e-tron', 'Q4 e-tron', '316', '320',
       '116', '120', '435', '118', '535', '328', '420', '418', '330',
       '428', '335', '318', '325', '218', 'M3', '640', '220', 'Z4', '650',
       'Z3', 'M4', 'X3', 'X1', 'X5', 'X6', 'X2', '523', '520', '525',
       '530', '740', '730', '225', '216', 'i3', 'i4', 'iX3', 'iX',
       'Fiesta', 'Focus', 'Puma', 'Mondeo', 'EcoSport', 'Fusion',
       'Ka/Ka+', 'Mustang', 'Focus CC', 'Explorer', 'Kuga', 'F 150',
       'C-Max', 'Focus C-Max', 'S-Max', 'Transit', 'Galaxy', 'B-Max',
       'Transit Connect', 'Ranger', 'Transit Custom', 'Mustang Mach-E',
       'A 180', 'A 45 AMG', 'A 160', 'A 200', 'A 250', 'GLA 45 AMG',
       'GLA 180', 'CLA 200', 'GLA 200', 'CLA 250', 'B 180', 'B 200',
       'GLA 250', 'A 150', 'A 170', 'E 200', 'SLK 200', 'SL 500',
       'CLK 200', 'E 250

registration

In [ ]:
df.registration.isna().sum()

38

In [ ]:
df[df.registration.isna()]

,location,province,make,model,make_model,price,body_type,registration,vehicle_age,mileage,...,engine_size,Fuel_type,upholstery_colour,doors,upholstery,Entertainment & Media,Comfort & Convenience,Safety,extras,general_inspection
12543,maastricht-airport,NaN,BMW,iX,BMW_iX,101861,Off-Road/Pick-up,NaN,NaN,10,...,1598,NaN,Other,5,Full leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Cruise control, Ele...","[['ABS', 'Adaptive Cruise Control', 'Alarm sys...","Alloy wheels,Automatically dimming interior mi...",NaN
12709,maastricht-airport,NaN,BMW,iX3,BMW_iX3,70261,Off-Road/Pick-up,NaN,NaN,10,...,1598,NaN,Other,5,Full leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Adaptive Cruise Control', 'Adaptive ...","Alloy wheels,Automatically dimming interior mi...",NaN
12710,maastricht-airport,NaN,BMW,iX3,BMW_iX3,70261,Off-Road/Pick-up,NaN,NaN,10,...,1598,NaN,Black,5,Full leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Adaptive Cruise Control', 'Adaptive ...","Alloy wheels,Automatically dimming interior mi...",NaN
12712,maastricht-airport,NaN,BMW,iX3,BMW_iX3,70261,Off-Road/Pick-up,NaN,NaN,10,...,1598,NaN,Black,5,Full leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Adaptive Cruise Control', 'Adaptive ...","Alloy wheels,Automatically dimming interior mi...",NaN
12713,maastricht-airport,NaN,BMW,iX3,BMW_iX3,70261,Off-Road/Pick-up,NaN,NaN,10,...,1598,NaN,Black,5,Full leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Adaptive Cruise Control', 'Adaptive ...","Alloy wheels,Automatically dimming interior mi...",NaN
14466,rotterdam,South Holland,Ford,Mustang,Ford_Mustang,116900,Coupe,NaN,NaN,65,...,5000,Gasoline,Other,2,Other,"Bluetooth, MP3, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock wi...","Alloy wheels,Shift paddles,Spoiler,Sport packa...",NaN
14543,rotterdam,South Holland,Ford,F 150,Ford_F 150,98900,Off-Road/Pick-up,NaN,NaN,50,...,5030,Gasoline,Black,4,Full leather,"Bluetooth, Digital radio, MP3, On-board comput...","360° camera, Air conditioning, Armrest, Automa...","[['ABS', 'Alarm system', 'Blind spot monitor',...","Alloy wheels,Automatically dimming interior mi...",NaN
14546,maasland,South Holland,Ford,F 150,Ford_F 150,129000,Off-Road/Pick-up,NaN,NaN,950,...,3496,Gasoline,Black,4,Full leather,"Bluetooth, CD player, Digital radio, Hands-fre...","360° camera, Air conditioning, Armrest, Automa...","[['ABS', 'Adaptive Cruise Control', 'Alarm sys...","All season tyres,Ambient lighting,Automaticall...",New
14550,rotterdam,South Holland,Ford,F 150,Ford_F 150,126900,Off-Road/Pick-up,NaN,NaN,,...,5030,Gasoline,Black,4,Other,"Bluetooth, Digital cockpit, Digital radio, On-...","360° camera, Air conditioning, Armrest, Cruise...","[['ABS', 'Alarm system', 'Blind spot monitor',...","Alloy wheels,Catalytic Converter,Spare tyre,Sp...",NaN
14551,rotterdam,South Holland,Ford,F 150,Ford_F 150,126900,Off-Road/Pick-up,NaN,NaN,,...,5030,Gasoline,Black,4,Other,"Bluetooth, Digital cockpit, Digital radio, On-...","360° camera, Air conditioning, Armrest, Cruise...","[['ABS', 'Alarm system', 'Blind spot monitor',...","Alloy wheels,Catalytic Converter,Spare tyre,Sp...",NaN


In [ ]:
now = datetime.datetime.now()
now.year

2022

In [ ]:
df.registration.fillna(now.year,inplace = True)

In [ ]:
df[df.registration=='2022']

,location,province,make,model,make_model,price,body_type,registration,vehicle_age,mileage,...,engine_size,Fuel_type,upholstery_colour,doors,upholstery,Entertainment & Media,Comfort & Convenience,Safety,extras,general_inspection
1149,harderwijk,Gelderland,Audi,A5,Audi_A5,58900,Convertible,2022,0.0,1,...,1984,Super 95,Black,2,Full leather,"Digital cockpit, Digital radio, Hands-free equ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...","Alloy wheels,Automatically dimming interior mi...",NaN
1860,angeren,Gelderland,Audi,Others,Audi_Others,225950,Off-Road/Pick-up,2022,0.0,1900,...,3996,Gasoline,Black,5,Part leather,"Digital cockpit, Digital radio, Hands-free equ...","Air conditioning, Air suspension, Armrest, Aut...","[['ABS', 'Adaptive Cruise Control', 'Adaptive ...","Alloy wheels,Automatically dimming interior mi...",NaN
1872,someren,North Brabant,Audi,SQ7,Audi_SQ7,199950,Off-Road/Pick-up,2022,0.0,3110,...,3996,Super 95,Black,5,Full leather,"Digital cockpit, Digital radio, Hands-free equ...","Air suspension, Auxiliary heating, Cruise cont...","[['Adaptive Cruise Control', 'Alarm system', '...","Alloy wheels,Automatically dimming interior mi...",NaN
1918,rijssen,Overijssel,Audi,Q2,Audi_Q2,45650,Off-Road/Pick-up,2022,0.0,1,...,999,Super 95,Black,5,Part leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['Adaptive Cruise Control', 'Central door loc...","Alloy wheels,Sport seats,Sport suspension",NaN
1921,apeldoorn,Gelderland,Audi,Q2,Audi_Q2,43450,Off-Road/Pick-up,2022,0.0,1,...,999,Super 95,Black,5,Part leather,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...","Alloy wheels,Sport seats,Sport suspension,Voic...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68468,hillegom,South Holland,Volvo,Others,Volvo_Others,61580,Off-Road/Pick-up,2022,0.0,1,...,1969,NaN,Black,5,Cloth,"Hands-free equipment, Induction charging for s...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...","Alloy wheels,Electronic parking brake,Spoiler",NaN
68472,meppel,Drenthe,Volvo,Others,Volvo_Others,60600,Off-Road/Pick-up,2022,0.0,10,...,1969,NaN,NaN,5,Other,"Android Auto, Apple CarPlay, Digital cockpit, ...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Adaptive Cruise Control', 'Adaptive ...","Alloy wheels,Automatically dimming interior mi...",NaN
68872,nijmegen,Gelderland,Volvo,XC40,Volvo_XC40,50728,Off-Road/Pick-up,2022,0.0,,...,1477,NaN,NaN,5,Cloth,"Digital cockpit, Digital radio, On-board compu...","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...","Alloy wheels,Ambient lighting,Automatically di...",New
69008,hillegom,South Holland,Volvo,V60,Volvo_V60,47595,Station wagon,2022,0.0,979,...,1969,NaN,Black,5,Cloth,On-board computer,"Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Automatically dimming interior mi...",NaN


We noticed that are new cars.

In [ ]:
df.registration.unique()

array(['2003', '2013', '2015', '2016', '2017', '2019', '2014', '2011',
       '2012', '2010', '2018', '2009', '2005', '2004', '2008', '2007',
       '2021', '2002', '2006', '2020', '2000', '2001', '2022', '1999',
       '2028', '1995', '2025', 2022, '1981', '1997', '1980', '1998',
       '1976', '1993', '1991', '1994', '1996'], dtype=object)

In [ ]:
df.registration = df.registration.astype(int)

vehicle_age

In [ ]:
df.vehicle_age.fillna(0,inplace=True)

In [ ]:
df.vehicle_age.value_counts(dropna=False)

 1.0     9144
 5.0     6198
 6.0     5759
 4.0     4768
 2.0     4681
 7.0     3925
 3.0     3556
 8.0     3189
 9.0     3173
 11.0    3145
 10.0    3135
 12.0    2890
 14.0    2370
 13.0    2343
 15.0    2014
 16.0    1715
 17.0    1646
 0.0     1613
 18.0    1374
 19.0    1088
 20.0     807
 21.0     549
 22.0     400
 25.0       6
 24.0       6
 23.0       5
 29.0       3
 27.0       2
 28.0       2
 26.0       2
-6.0        1
-3.0        1
 41.0       1
 42.0       1
 46.0       1
 31.0       1
Name: vehicle_age, dtype: int64

In [ ]:
df[df.vehicle_age<0].index

Int64Index([2401, 5773], dtype='int64')

In [ ]:
df.drop(df[df.vehicle_age<0].index,axis = 0,inplace =True)

In [ ]:
df.reset_index(drop=True,inplace =True)

In [ ]:
df.vehicle_age = df.vehicle_age.astype(int)

fuel_type


In [ ]:
df.fuel_type.unique()

array(['Gasoline', 'Diesel', 'Electric/Diesel', 'LPG', 'Electric',
       'Electric/Gasoline', 'CNG', 'Ethanol', 'Others', 'Hydrogen'],
      dtype=object)

drivetrain

In [ ]:
df.drivetrain.unique()

array(['Front', nan, '4WD', 'Rear'], dtype=object)

In [ ]:
df.drivetrain.replace({'None':np.nan},inplace=True)

In [ ]:
df.drivetrain.value_counts(dropna = False)

Front    37203
NaN      16157
4WD       8108
Rear      8044
Name: drivetrain, dtype: int64

In [ ]:
fill(df,'drivetrain','mode','make_model','body_type')

last Nan values filled only group1
Number of NaN :  0
------------------
Front    49896
Rear     10456
4WD       9160
Name: drivetrain, dtype: int64
Number of NaN :  0
------------------
Front    49896
Rear     10456
4WD       9160
Name: drivetrain, dtype: int64


In [ ]:
df.groupby(['make_model','body_type']).agg(pd.Series.mode)

location  \
make_model body_type                                                    
Audi_A1    Compact                               [apeldoorn, tilburg]   
           Coupe                                      kootwijkerbroek   
           Off-Road/Pick-up                                    zwolle   
           Other                                         hoogblokland   
           Sedan                                              helmond   
...                                                               ...   
Volvo_XC90 Off-Road/Pick-up                                 apeldoorn   
           Other             [amsterdam, groningen, lienden, utrecht]   
           Station wagon                              's gravenpolder   
           Transporter                                          emmen   
           Van                                                  zeist   

                                                                    province  \
make_model body_type                                                           
Audi_A1    Compact                                                Gelderland   
           Coupe                                                  Gelderland   
           Off-Road/Pick-up                                       Gelderland   
           Other                                               South Holland   
           Sedan                                               North Brabant   
...                                                                      ...   
Volvo_XC90 Off-Road/Pick-up                                       Gelderland   
           Other             [Gelderland, Groningen, North Holland, Utrecht]   
           Station wagon                                                  []   
           Transporter                                               Drenthe   
           Van                                                 North Brabant   

                              make model  \
make_model body_type                       
Audi_A1    Compact            Audi    A1   
           Coupe              Audi    A1   
           Off-Road/Pick-up   Audi    A1   
           Other              Audi    A1   
           Sedan              Audi    A1   
...                            ...   ...   
Volvo_XC90 Off-Road/Pick-up  Volvo  XC90   
           Other             Volvo  XC90   
           Station wagon     Volvo  XC90   
           Transporter       Volvo  XC90   
           Van               Volvo  XC90   

                                                               price  \
make_model body_type                                                   
Audi_A1    Compact                                              8999   
           Coupe                                               10950   
           Off-Road/Pick-up             [35250, 35450, 36350, 39950]   
           Other                                               24995   
           Sedan                                                8750   
...                                                              ...   
Volvo_XC90 Off-Road/Pick-up                                    49950   
           Other                         [8999, 26950, 46750, 65900]   
           Station wagon                                        5450   
           Transporter       [5950, 7750, 9250, 10250, 12950, 23950]   
           Van                                                  8500   

                                                     registration  \
make_model body_type                                                
Audi_A1    Compact                                           2011   
           Coupe                                             2011   
           Off-Road/Pick-up                                  2021   
           Other                                             2020   
           Sedan                                             2011   
...                                                           ...   
Volvo_XC

non_smoker_vehicle

In [ ]:
df.non_smoker_vehicle.value_counts()

No     63903
Yes     5609
Name: non_smoker_vehicle, dtype: int64

empty_weight

In [ ]:
df.empty_weight.dtype

dtype('O')

In [ ]:
df.empty_weight= df.empty_weight.astype('float')

In [ ]:
df.empty_weight[df.empty_weight<550]

615      180.0
11273    112.0
37639    376.0
38611    276.0
38649    276.0
38650    474.0
38686    276.0
38807    375.0
38808    474.0
38809    276.0
38810    376.0
50232      1.0
63754      1.0
Name: empty_weight, dtype: float64

In [ ]:
df.loc[(df.empty_weight < 550 ),'empty_weight']=np.nan

In [ ]:
df.empty_weight[df.empty_weight<550]

Series([], Name: empty_weight, dtype: float64)

In [ ]:
df.empty_weight.isna().sum()

1588

In [ ]:
fill(df,'empty_weight','median','make_model','body_type')

last Nan values filled only group1
Number of NaN :  0
------------------
1295.0    1755
1405.0    1293
1635.0     891
1065.0     757
1515.0     732
          ... 
2373.0       1
1347.5       1
1534.0       1
1183.0       1
2147.0       1
Name: empty_weight, Length: 1589, dtype: int64


In [ ]:
df.empty_weight

0        1280.0
1        1200.0
2        1200.0
3        1085.0
4        1105.0
          ...  
69507    1441.0
69508    1489.0
69509    1495.0
69510    1290.0
69511    1488.0
Name: empty_weight, Length: 69512, dtype: float64

In [ ]:
df.empty_weight.isna().sum()

0

In [ ]:
df.empty_weight = df.empty_weight.astype(int)

In [ ]:
df.empty_weight = df.empty_weight.apply(lambda x : re.sub(r'\.0','',str(x)))


In [ ]:
df.empty_weight = df.empty_weight.astype(int)

Paint


In [ ]:
df.paint

0         Metallic
1              NaN
2         Metallic
3         Metallic
4              NaN
           ...    
69507          NaN
69508     Metallic
69509     Metallic
69510     Metallic
69511     Metallic
Name: paint, Length: 69512, dtype: object

In [ ]:
df.paint.replace({'None':np.nan},inplace = True)

In [ ]:
df.paint.value_counts(dropna=False)

 Metallic    49115
NaN          20397
Name: paint, dtype: int64

In [ ]:
df.drop('paint',axis = 1 , inplace = True)

province

In [ ]:
df.province.value_counts(dropna=False)

Gelderland       12416
North Brabant    11261
South Holland     9048
North Holland     7520
Utrecht           6091
NaN               4652
Overijssel        3890
Limburg           3678
Drenthe           3301
Friesland         3153
Flevoland         2123
Groningen         1445
Zeeland            934
Name: province, dtype: int64

In [ ]:
df.province.fillna(method = 'ffill',inplace=True)

In [ ]:
df.province.value_counts()

Gelderland       13222
North Brabant    12113
South Holland     9724
North Holland     8076
Utrecht           6540
Overijssel        4178
Limburg           3941
Drenthe           3536
Friesland         3345
Flevoland         2258
Groningen         1575
Zeeland           1004
Name: province, dtype: int64

Droped useless feature

In [ ]:
df.drop('location',axis=1 , inplace=True)

In [ ]:
#df.drop('model',axis=1 , inplace=True)

# MUHAMMED

1 milage


In [ ]:
df["mileage"] = pd.to_numeric(df["mileage"])

In [ ]:
fill(df,'mileage', 'median','make_model', 'vehicle_age')

last Nan values filled only group1
Number of NaN :  0
------------------
10.0        1576
1.0         1161
5.0          834
15.0         798
12.0         168
            ... 
62905.0        1
136195.0       1
117819.0       1
117950.0       1
404758.0       1
Name: mileage, Length: 49609, dtype: int64


In [ ]:
df[df['mileage'].isnull()].mileage

Series([], Name: mileage, dtype: float64)

In [ ]:
df.mileage.fillna(df.mileage.median(), inplace=True)
df.mileage.isnull().sum()

0

In [ ]:
df['mileage'] = df["mileage"].astype(int)

2 co2_emissions

In [ ]:
df["co2_emissions"] = pd.to_numeric(df["co2_emissions"])

In [ ]:
# elektrikli ve hidrojenli araclarin emilsiyon degeri 0'a esitlendi
# fosil yakitlarin 0 olan degerleri nan yapildi
df.co2_emissions.loc[(df['fuel_type'] == 'Electric') | (df['fuel_type'] == 'Hydrogen')] = 0
df.co2_emissions.loc[(df['fuel_type'] != 'Electric') & (df['fuel_type'] != 'Hydrogen')]=df.co2_emissions.loc[(df['fuel_type'] != 'Electric') & (df['fuel_type'] != 'Hydrogen')].replace(to_replace=0.0, value=np.nan)

In [ ]:
fill(df,'co2_emissions', 'median', 'make_model', 'fuel_type')

last Nan values filled only group1
Number of NaN :  0
------------------
0.0      5547
139.0    1751
119.0    1514
149.0    1206
159.0    1081
         ... 
332.0       1
71.0        1
55.0        1
142.5       1
400.0       1
Name: co2_emissions, Length: 403, dtype: int64


In [ ]:
df['co2_emissions'] = df["co2_emissions"].astype(int)

4 doors

In [ ]:
df["doors"] = pd.to_numeric(df["doors"])

In [ ]:
df.doors.isnull().sum()

1310

In [ ]:
fill(df,'doors', 'ffill','make_model', 'body_type')

last Nan values filled with columns
Number of NaN :  0
------------------
5      51163
4       9753
2       5877
3       2618
6         97
1          4
NaN        0
Name: doors, dtype: Int64


In [ ]:
df['doors'] = df["doors"].astype(int)

5 gears

In [ ]:
df["gears"] = pd.to_numeric(df["gears"])

In [ ]:
df.gears.isnull().sum()

20252

In [ ]:
fill(df, 'gears', 'ffill','make_model', 'body_type')

last Nan values filled with columns
Number of NaN :  0
------------------
6.0    27416
5.0    16035
7.0    10797
8.0     7563
1.0     5332
9.0     1226
4.0     1008
2.0       94
3.0       41
Name: gears, dtype: int64


In [ ]:
df['gears'] = df["gears"].astype(int)

6 colour

In [ ]:
fill(df,  'colour', 'ffill','make_model', 'body_type')

last Nan values filled with columns
Number of NaN :  0
------------------
Grey      23258
Black     17908
White     10182
Blue       8804
Red        3218
Brown      1903
Silver     1850
Green       884
Beige       569
Orange      371
Yellow      301
Violet      170
Gold         58
Bronze       36
Name: colour, dtype: int64


7 upholstery

In [ ]:
fill(df, 'upholstery', 'ffill', 'make_model', 'body_type')

last Nan values filled with columns
Number of NaN :  0
------------------
Cloth           32644
Full leather    21843
Part leather    12307
alcantara        1403
Other             689
Velour            626
Name: upholstery, dtype: int64


8 combination(L/100Km)

In [ ]:
df["combination(L/100Km)"] = pd.to_numeric(df["combination(L/100Km)"])

In [ ]:
df['combination(L/100Km)'].isna().sum()

21166

In [ ]:
df['combination(L/100Km)'].loc[(df['combination(L/100Km)'] < 3) | (df['combination(L/100Km)'] > 18 )]=np.nan

In [ ]:
df['combination(L/100Km)'].isna().sum()

21731

In [ ]:
fill(df, 'combination(L/100Km)', 'median','make_model', 'fuel_type')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
6.000     6456
5.200     2242
5.900     2119
5.100     2091
6.400     1987
          ... 
4.825        1
16.500       1
15.600       1
15.700       1
9.950        1
Name: combination(L/100Km), Length: 176, dtype: int64


In [ ]:
df['combination(L/100Km)'].loc[(df['fuel_type'] == 'Electric') | (df['fuel_type'] == 'Hydrogen')] = 0
df['combination(L/100Km)'] = df["combination(L/100Km)"].astype(int)

9 city(L/100Km)

In [ ]:
df["city(L/100Km)"] = pd.to_numeric(df["city(L/100Km)"])

In [ ]:
df['city(L/100Km)'].loc[(df['city(L/100Km)'] < 3) | (df['city(L/100Km)'] > 18 )]=np.nan

In [ ]:
fill(df, 'city(L/100Km)', 'median','make_model', 'fuel_type')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
7.50     5586
6.60     1533
6.30     1461
6.80     1458
6.00     1434
         ... 
5.35        2
12.10       1
14.65       1
4.95        1
3.00        1
Name: city(L/100Km), Length: 190, dtype: int64


In [ ]:
df['city(L/100Km)'].loc[(df['fuel_type'] == 'Electric') | (df['fuel_type'] == 'Hydrogen')] = 0
df['city(L/100Km)'] = df['city(L/100Km)'].astype(int)

10 country(L/100Km)

In [ ]:
df["country(L/100Km)"] = pd.to_numeric(df["country(L/100Km)"])

In [ ]:
# yakit tuketimi 3 ile 18 arasinda olmasi beklenir
df['country(L/100Km)'].loc[(df['country(L/100Km)'] < 3) | (df['country(L/100Km)'] > 18 )]=np.nan

In [ ]:
fill(df, 'country(L/100Km)', 'median','make_model', 'fuel_type')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
5.100     6928
4.500     3582
4.700     3165
4.900     2916
5.400     2530
          ... 
12.800       1
11.300       1
13.800       1
17.400       1
6.425        1
Name: country(L/100Km), Length: 143, dtype: int64


In [ ]:
df['country(L/100Km)'].loc[(df['fuel_type'] == 'Electric') | (df['fuel_type'] == 'Hydrogen')] = 0
df['country(L/100Km)'] = df['country(L/100Km)'].astype(int)

# Efe

### general_inspection

In [ ]:
today= str(datetime.datetime.now().strftime("%m/%Y")) #need current day ,forcomparing whether valid inspection date 
today

'04/2022'

In [ ]:
df.general_inspection.replace({'None': np.nan,'New': today},inplace = True)

In [ ]:
#changes inspection date to today,for new cars (from type column)
df['general_inspection'] = np.where((df.type =="New"),today,df.general_inspection) 

In [ ]:
# df["general_inspection"]=pd.to_datetime(df.general_inspection)

In [ ]:
df['general_inspection']=(df['general_inspection'] >= today)


In [ ]:
df['general_inspection'].value_counts()

True     48121
False    21391
Name: general_inspection, dtype: int64

### cylinders

In [ ]:
df['cylinders'].head()

0    4
1    4
2    4
3    4
4    4
Name: cylinders, dtype: object

In [ ]:
df['cylinders'].value_counts(dropna=False)

4      46546
3       8482
NaN     6182
6       4661
5       1531
8       1024
1        554
2        433
10        72
12        25
7          2
Name: cylinders, dtype: int64

In [ ]:
fill_prop(df,'cylinders','make_model','body_type')

Number of NaN :  371
------------------
4      47855
3       8852
6       4740
1       4427
5       1585
8       1127
2        453
NaN      371
10        75
12        25
7          2
Name: cylinders, dtype: int64


In [ ]:
fill_most(df,'cylinders','make_model','body_type')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
4     48150
3      8858
6      4746
1      4491
5      1585
8      1127
2       453
10       75
12       25
7         2
Name: cylinders, dtype: int64


In [ ]:
df['cylinders']=df['cylinders'].astype(int)

### seats

In [ ]:
df.seats.replace({'': np.nan},inplace = True)


In [ ]:
df["seats"].value_counts(dropna=False)


5      51852
4       6604
NaN     5442
2       2183
7       2025
3        991
6        286
8         64
9         63
1          2
Name: seats, dtype: int64

In [ ]:
fill_most(df,"seats","make_model")

Number of NaN :  0
------------------
5    56243
4     7020
2     2516
7     2128
3     1189
6      287
8       64
9       63
1        2
Name: seats, dtype: int64


In [ ]:
fill_most(df,"seats","make_model",'body_type')

Number of NaN :  0
------------------
5    56243
4     7020
2     2516
7     2128
3     1189
6      287
8       64
9       63
1        2
Name: seats, dtype: int64


In [ ]:
df["seats"]= df["seats"].astype(int)

In [ ]:
df["seats"].value_counts(dropna=False)

5    56243
4     7020
2     2516
7     2128
3     1189
6      287
8       64
9       63
1        2
Name: seats, dtype: int64

### upholstery_colour

In [ ]:
df.upholstery_colour.replace({'None': np.nan},inplace = True)

In [ ]:
df["upholstery_colour"].value_counts(dropna=False)

Black     33611
NaN       24278
Grey       6000
Beige      1721
Other      1665
Brown      1249
Red         324
Blue        321
White       289
Orange       24
Yellow       17
Green        13
Name: upholstery_colour, dtype: int64

In [ ]:
fill(df,"upholstery_colour","ffill","make_model","body_type")

last Nan values filled with columns
Number of NaN :  0
------------------
Black     50971
Grey      10067
Beige      2766
Other      2299
Brown      1830
Blue        537
Red         518
White       440
Orange       37
Yellow       28
Green        19
Name: upholstery_colour, dtype: int64


### warranty

In [ ]:
df["warranty_months"]=df["warranty_months"].astype(int)

In [ ]:
df["warranty_months"].value_counts(dropna=False)

0     45995
6     18502
3      1810
12     1490
24     1152
1       262
84      231
18       33
36       13
2         8
48        7
60        3
26        2
9         2
10        1
4         1
Name: warranty_months, dtype: int64

### previous_owner , Fuel_type ,seller

In [ ]:
df["previous_owner"].value_counts()

      64665
1      2954
2      1299
3       386
4       132
5        41
6        14
7         8
8         7
9         2
10        2
14        1
12        1
Name: previous_owner, dtype: int64

In [ ]:
#+%90 missing
df.drop(['previous_owner'], axis=1, inplace=True) 

In [ ]:
#double column
df.drop(['Fuel_type'], axis=1, inplace=True) 

In [ ]:
#no any nan in seller column
df["seller"].value_counts(dropna=False)

Dealer            63454
Private seller     6058
Name: seller, dtype: int64

# Serkan

In [ ]:
df['make'].value_counts()

BMW              7647
Volkswagen       7193
Mercedes-Benz    6547
Audi             6244
Volvo            5168
Renault          4732
Ford             4469
Peugeot          3885
Toyota           3081
Skoda            2857
Kia              2812
Opel             2583
Hyundai          2520
Mazda            2413
Citroen          2395
Fiat             2080
Honda            1052
Dacia             908
Chevrolet         469
Tesla             457
Name: make, dtype: int64

In [ ]:
df['make'].isna().sum()

0

In [ ]:
df['Gearbox'].head()

0    Automatic
1    Automatic
2    Automatic
3    Automatic
4    Automatic
Name: Gearbox, dtype: object

In [ ]:
fill_most(df,'Gearbox','make_model','body_type')

Number of NaN :  0
------------------
Automatic         40638
Manual            28180
Semi-automatic      694
Name: Gearbox, dtype: int64


In [ ]:
df['Gearbox'].value_counts()

Automatic         40638
Manual            28180
Semi-automatic      694
Name: Gearbox, dtype: int64

In [ ]:
df['Gearbox'].isnull().sum()

0

In [ ]:
df['emission_class(Euro)'].head()


0      4
1    NaN
2    NaN
3    NaN
4      6
Name: emission_class(Euro), dtype: object

In [ ]:
df['emission_class(Euro)'].value_counts(dropna=False)

6      24869
NaN    21856
5      11752
4       8475
3       1891
1        520
2        149
Name: emission_class(Euro), dtype: int64

In [ ]:
fill_most(df,'emission_class(Euro)','registration')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
6    39857
5    14968
4    11628
3     2380
1      520
2      159
Name: emission_class(Euro), dtype: int64


In [ ]:
df['emission_class(Euro)'].isnull().sum()

0

In [ ]:
df["emission_class(Euro)"]=df["emission_class(Euro)"].astype(int)

In [ ]:
df['type'].head()

0    Used
1    Used
2    Used
3    Used
4    Used
Name: type, dtype: object

In [ ]:
df['type'].value_counts(dropna=False)

Used                 61455
New                   3846
Demonstration         2038
Pre-registered        1101
NaN                   1071
Antique / Classic        1
Name: type, dtype: int64

In [ ]:
fill_prop(df,'type','registration')

Number of NaN :  1
------------------
Used                 62497
New                   3860
Demonstration         2045
Pre-registered        1108
Antique / Classic        1
NaN                      1
Name: type, dtype: int64


In [ ]:
df.drop(df[df['type'].isna()].index,inplace=True)


In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df['type'].isna().sum()

0

In [ ]:
df[df['type'].isna()].index

Int64Index([], dtype='int64')

In [ ]:
df['full_service_history'].head()

0    Yes
1    Yes
2    NaN
3    Yes
4    Yes
Name: full_service_history, dtype: object

In [ ]:
df['full_service_history']=df['full_service_history'].fillna('No')

In [ ]:
df['full_service_history'].value_counts(dropna=False)

Yes    47556
No     21955
Name: full_service_history, dtype: int64

In [ ]:
df

,province,make,model,make_model,price,body_type,registration,vehicle_age,mileage,Gearbox,...,warranty_months,engine_size,upholstery_colour,doors,upholstery,Entertainment & Media,Comfort & Convenience,Safety,extras,general_inspection
0,North Brabant,Audi,A3,Audi_A3,4450,Compact,2003,19,191066,Automatic,...,6,1984,Black,3,Cloth,"CD player, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...",Alloy wheels,True
1,Limburg,Audi,A3,Audi_A3,12950,Compact,2013,9,94758,Automatic,...,0,1395,Black,5,Cloth,"CD player, Radio","Air conditioning, Automatic climate control, E...","[['Alarm system', 'Central door lock', 'Centra...",Alloy wheels,False
2,North Brabant,Audi,A3,Audi_A3,12445,Compact,2013,9,160883,Automatic,...,0,1395,Black,5,Cloth,"Bluetooth, CD player, Hands-free equipment, MP...","Air conditioning, Armrest, Cruise control, Ele...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Spoiler",False
3,North Brabant,Audi,A1,Audi_A1,12450,Compact,2015,7,145102,Automatic,...,6,1395,Black,5,Part leather,"Bluetooth, CD player, On-board computer, USB","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock with remote contro...",Alloy wheels,True
4,Gelderland,Audi,A1,Audi_A1,14945,Compact,2015,7,119666,Automatic,...,6,1395,Black,3,Part leather,"CD player, MP3, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Bi-Xenon headlights'...","Alloy wheels,Shift paddles,Sport seats,Sport s...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69506,South Holland,Volvo,S60,Volvo_S60,1999,Sedan,2005,17,442713,Manual,...,0,2435,Beige,4,Part leather,"CD player, On-board computer, Radio","Air conditioning, Automatic climate control, C...","[['ABS', 'Central door lock', 'Central door lo...",Alloy wheels,True
69507,Friesland,Volvo,S60,Volvo_S60,3950,Sedan,2005,17,251723,Manual,...,0,2435,Beige,4,Full leather,Radio,"Air conditioning, Automatic climate control, C...","[['ABS', 'Alarm system', 'Central door lock', ...",Alloy wheels,True
69508,Flevoland,Volvo,S60,Volvo_S60,3449,Sedan,2004,18,340257,Manual,...,0,1984,Beige,4,Part leather,On-board computer,"Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Automatically dimming interior mi...",True
69509,North Holland,Volvo,S40,Volvo_S40,1650,Sedan,2004,18,399990,Manual,...,0,2435,Beige,4,Velour,"CD player, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock with remote contro...","Alloy wheels,Automatically dimming interior mi...",False


In [ ]:
df['full_service_history'].isnull().sum()

0

In [ ]:
df['power(kW)'].head()

0    110
1     92
2     90
3     92
4     92
Name: power(kW), dtype: object

In [ ]:
df['power(kW)'].value_counts()

       5979
110    4082
88     2806
100    2479
90     2420
       ... 
372       1
371       1
208       1
237       1
10        1
Name: power(kW), Length: 366, dtype: int64

In [ ]:
df['power(kW)'].replace({'': np.nan},inplace=True)

In [ ]:
df['power(kW)'].isnull().sum()

5979

In [ ]:
fill_most(df,'power(kW)','make_model','body_type')

last Nan values filled only group1
Number of NaN :  0
------------------
110    4978
88     2884
100    2859
90     2587
85     2357
       ... 
372       1
371       1
208       1
237       1
10        1
Name: power(kW), Length: 365, dtype: int64


In [ ]:
df['power(kW)'].isnull().sum()

0

In [ ]:
df['power(kW)'] = df['power(kW)'].astype(int)

In [ ]:
df['power(kW)'] = np.where((df['power(kW)']<50),np.nan,df['power(kW)'])

In [ ]:
df['power(kW)'].isnull().sum()

700

In [ ]:
fill_most(df,'power(kW)','make_model','body_type')

last Nan values filled only group1
last Nan values filled with columns
Number of NaN :  0
------------------
110.0    5039
88.0     2886
100.0    2861
90.0     2588
85.0     2357
         ... 
328.0       1
319.0       1
303.0       1
277.0       1
189.0       1
Name: power(kW), Length: 337, dtype: int64


In [ ]:
df['Comfort & Convenience'].head()

0    Air conditioning, Armrest, Automatic climate c...
1    Air conditioning, Automatic climate control, E...
2    Air conditioning, Armrest, Cruise control, Ele...
3    Air conditioning, Armrest, Automatic climate c...
4    Air conditioning, Armrest, Automatic climate c...
Name: Comfort & Convenience, dtype: object

In [ ]:
df['Comfort & Convenience'].isnull().sum()

0

In [ ]:
df.isna().sum()

province                 0
make                     0
model                    0
make_model               0
price                    0
body_type                0
registration             0
vehicle_age              0
mileage                  0
Gearbox                  0
gears                    0
seller                   0
colour                   0
power(kW)                0
cylinders                0
drivetrain               0
type                     0
full_service_history     0
fuel_type                0
combination(L/100Km)     0
city(L/100Km)            0
country(L/100Km)         0
non_smoker_vehicle       0
empty_weight             0
co2_emissions            0
emission_class(Euro)     0
seats                    0
warranty_months          0
engine_size              0
upholstery_colour        0
doors                    0
upholstery               0
Entertainment & Media    0
Comfort & Convenience    0
Safety                   0
extras                   0
general_inspection       0
d

In [ ]:
df.dtypes

province                  object
make                      object
model                     object
make_model                object
price                      int64
body_type                 object
registration               int64
vehicle_age                int64
mileage                    int64
Gearbox                   object
gears                      int64
seller                    object
colour                    object
power(kW)                float64
cylinders                  int64
drivetrain                object
type                      object
full_service_history      object
fuel_type                 object
combination(L/100Km)       int64
city(L/100Km)              int64
country(L/100Km)           int64
non_smoker_vehicle        object
empty_weight               int64
co2_emissions              int64
emission_class(Euro)       int64
seats                      int64
warranty_months            int64
engine_size                int64
upholstery_colour         object
doors     

In [ ]:
df.to_pickle('/content/drive/MyDrive/Colab Notebooks/withoutNAN.pickle')

In [ ]:
df

,province,make,model,make_model,price,body_type,registration,vehicle_age,mileage,Gearbox,...,warranty_months,engine_size,upholstery_colour,doors,upholstery,Entertainment & Media,Comfort & Convenience,Safety,extras,general_inspection
0,North Brabant,Audi,A3,Audi_A3,4450,Compact,2003,19,191066,Automatic,...,6,1984,Black,3,Cloth,"CD player, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Central door lock', ...",Alloy wheels,True
1,Limburg,Audi,A3,Audi_A3,12950,Compact,2013,9,94758,Automatic,...,0,1395,Black,5,Cloth,"CD player, Radio","Air conditioning, Automatic climate control, E...","[['Alarm system', 'Central door lock', 'Centra...",Alloy wheels,False
2,North Brabant,Audi,A3,Audi_A3,12445,Compact,2013,9,160883,Automatic,...,0,1395,Black,5,Cloth,"Bluetooth, CD player, Hands-free equipment, MP...","Air conditioning, Armrest, Cruise control, Ele...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Spoiler",False
3,North Brabant,Audi,A1,Audi_A1,12450,Compact,2015,7,145102,Automatic,...,6,1395,Black,5,Part leather,"Bluetooth, CD player, On-board computer, USB","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock with remote contro...",Alloy wheels,True
4,Gelderland,Audi,A1,Audi_A1,14945,Compact,2015,7,119666,Automatic,...,6,1395,Black,3,Part leather,"CD player, MP3, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Alarm system', 'Bi-Xenon headlights'...","Alloy wheels,Shift paddles,Sport seats,Sport s...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69506,South Holland,Volvo,S60,Volvo_S60,1999,Sedan,2005,17,442713,Manual,...,0,2435,Beige,4,Part leather,"CD player, On-board computer, Radio","Air conditioning, Automatic climate control, C...","[['ABS', 'Central door lock', 'Central door lo...",Alloy wheels,True
69507,Friesland,Volvo,S60,Volvo_S60,3950,Sedan,2005,17,251723,Manual,...,0,2435,Beige,4,Full leather,Radio,"Air conditioning, Automatic climate control, C...","[['ABS', 'Alarm system', 'Central door lock', ...",Alloy wheels,True
69508,Flevoland,Volvo,S60,Volvo_S60,3449,Sedan,2004,18,340257,Manual,...,0,1984,Beige,4,Part leather,On-board computer,"Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock', 'Central door lo...","Alloy wheels,Automatically dimming interior mi...",True
69509,North Holland,Volvo,S40,Volvo_S40,1650,Sedan,2004,18,399990,Manual,...,0,2435,Beige,4,Velour,"CD player, On-board computer, Radio","Air conditioning, Armrest, Automatic climate c...","[['ABS', 'Central door lock with remote contro...","Alloy wheels,Automatically dimming interior mi...",False
